# Misc

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
import scipy as sp
import pylab as py
import pandas as pd
import mpl_toolkits.mplot3d.axes3d as p3
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
import matplotlib.animation as animation
from pytreegrav import Accel, Potential
# from ipynb.fs.full.Neutron_Star_SPH_sph import *
# from ipynb.fs.full.Neutron_Star_SPH_eos import *
#from ipynb.fs.full.simulate import *

In [ ]:
#@title Define plotting functions

def plot_single_timestep(data, timestep=None, title="Star timestep data"):
    """
    data -- numpy array containing single timestep data or a collection of timesteps.
    Timestep -- the indice to index the data to aquire the proper timestep.
    This function only updates the plot; plt.show() has to be called after it
    """
    str_=""
    if timestep!=None:
        data = data[timestep]
        str_ = " at timestep "+ str(timestep)
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.set_title(title+str_)
    ax.set_xlabel('km')
    ax.set_ylabel('km')
    ax.set_zlabel('km')
    scatter = ax.scatter(data[:,0], data[:,1], data[:,2], s=1)
    return fig, ax, scatter

In [ ]:
def interpolate(val, x,y):
    return np.interp(val, x, y)

def update_dAdt(star, init = False):
    """ Initializes dAdt if necessary
        Otherwise, calculates dAdt from the star's properties
    """
    dAdt = 0
    if init:  # must initialze dAdt by the star's other properties
        return np.zeros(np.shape(star.pos))
    return dAdt

def calculate_tstep(star):
    return 1e-5

def calculate_div(star):
    """ divergence/curl of the velocity field. """
    d = star.vel
    c = star.vel
    return d,c # returns divergence and curl of the velocity field

def calculate_h(star, init=False):
    if init: # must initialze h by the star's other properties
        h = star.eta*(star.rho/star.m)**(1/3)
        star.h = h
    else:
        h = star.eta*(star.rho/star.m)**(1/3)
        star.h = h
    return star.h

def gravity_components(star):
    """Takes a set of x,yz coordinate positions and returns x,y,z accelerations due to gravity"""
    N = np.shape(star.pos)[0]
    h = np.repeat(0.01,N)
    mass = np.asarray([1/N for i in range(N)]) # make the system have unit mass.
    return Accel(star.pos, mass,h)
    #print(Potential(star.pos,star.m))    

def init_xva(N, R = None,random=True): # N = numpoints
    """
    Initializes position and velocity, and acceleration (random)
    """
    if R == None:
        print("Need to specify radius in initialization function.")
    if random:
        np.random.seed(42) # set the random number generator seed ============= Initial pos and veloc ===
        pos = np.random.randn(N,3)*R   # (km) positions of the particles relative to the center of mass (in km)         # randomly select positions and velocities from initialized seed
        pos = np.random.uniform(low=-R, high=R, size=(N,3))
        #print(pos)
        vel = np.zeros(pos.shape)    # (km/s) randomly select positions and velocities positions and velocities are N x 3 matrices.
        acc = np.zeros(pos.shape)  # (km/s^2) initially we haven't updated the accelerations of those random points for the star.
        return pos, vel, acc
    if not random:
        pos = mesh_pts(N,R)   # (km) positions of the particles relative to the center of mass (in km)         # randomly select positions and velocities from initialized seed
        vel = np.zeros(pos.shape)    # (km/s) randomly select positions and velocities positions and velocities are N x 3 matrices.
        acc = np.zeros(pos.shape) 
    return pos,vel,acc

In [3]:
import numpy as np
from pytreegrav import Accel, Potential

N = 10**5 # number of particles
x = np.random.rand(N,3) # positions randomly sampled in the unit cube
m = np.repeat(1./N,N) # masses - let the system have unit mass
h = np.repeat(0.01,N) # softening radii - these are optional, assumed 0 if not provided to the frontend functions
#print(Accel(x,m))
#print(Potential(x,m))

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/alexastewart/opt/anaconda3/lib/python3.9/site-packages/pytreegrav/treewalk.py:305: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  set_parallel_chunksize(10000)


[[-0.24321304  1.50239701  0.40920213]
 [-1.207051    0.36851447 -0.13960535]
 [-0.47894326 -0.3910418  -1.80234003]
 ...
 [ 0.19604723 -0.16242835  0.15268837]
 [ 0.94484886 -1.57111233 -0.32955978]
 [-1.3365983  -1.62370671 -0.26063592]]
[-2.0994771  -2.21882878 -1.98069632 ... -2.37242315 -1.92120053
 -1.47815844]


In [9]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 27 21:40:45 2014
credit: Mohammad Asif Zaman
"""
def mesh_pts(N,R):
    """ generates N particles inside a sphere mapped onto a uniform grid """
    R = R**2
    val = int(np.sqrt(N)) + 3
    mesh = np.indices((val,val,val)).T.reshape(-1,3)
    r = mesh[0,:]**2 + mesh[1,:]**2 + mesh[2,:]**2
    indices = np.where(r<=R)
    np.delete(mesh, indices, None)
    return mesh

import time
import math
import numpy as np
import scipy as sp

hc = 197.327                # Conversion factor in MeV fm (hut * c)
G = hc * 6.67259e-45        # Gravitational constant
Ms = 1.1157467e60
rho_s = 1665.3              # Central density (density at r = 0)
M0 = (4*3.14159265*(G**3)*rho_s)**(-0.5)
R0 = G*M0
mn = 938.926   

def initial_n():          
    n = 1
    err = 1
    tol = 1e-15
    count = 0
    # Newton-Raphson method
    while err > tol : 
        count += 1
        fn = n*mn + 236*n**(2.54) - rho_s
        dfn = mn + 236*2.54*n**(1.54)
        temp = n - fn/dfn
        err = np.abs(n-temp)
        n = temp
    print("Newton-Raphson Converged after ", count, "iterations")
    return n


def rho(p):
    n = (p*rho_s/363.44)**(1./2.54)
    return (236. * n**2.54 + n *mn)/rho_s 


def dp_dr(r,m,p,flag):
    if flag == 0:                              # classical model
        y = -m*rho(p)/(r**2 + 1e-20)
    else:                                      # relativistic model
        rh = rho(p)                            
        y = -(p+rh)*(p*r**3 + m)/(r**2 - 2*m*r + 1e-20)
    return y

def dm_dr(r,m,p):
    return rho(p)*r**2


def EulerSolver(r,m,p,h,flag):
    y = np.zeros(2)
    y[0] = m + dm_dr(r,m,p)*h
    y[1] = p + dp_dr(r,m,p,flag)*h
    return y

def RK4Solver(r,m,p,h,flag):
    y = np.zeros(2)
    k11 = dm_dr(r,m,p)
    k21 = dp_dr(r,m,p,flag)

    k12 = dm_dr(r+0.5*h,m+0.5*k11*h,p+0.5*k21*h)
    k22 = dp_dr(r+0.5*h,m+0.5*k11*h,p+0.5*k21*h,flag)

    k13 = dm_dr(r+0.5*h,m+0.5*k12*h,p+0.5*k22*h)
    k23 = dp_dr(r+0.5*h,m+0.5*k12*h,p+0.5*k22*h,flag)

    k14 = dm_dr(r+h,m+h*k13,p+h*k23)    
    k24 = dp_dr(r+h,m+h*k13,p+h*k23,flag)    

    y[0] = m + h*(k11 + 2.*k12 + 2.*k13 + k14)/6.
    y[1] = p + h*(k21 + 2.*k22 + 2.*k23 + k24)/6.
    return y

def pressure_ross2018(rho,L,r,eos):
    """ rho is either density (scalar) or a matrix with density as its first element."""
    pls = P_at_r(r,eos.interpolate)
    rls = rho_at_r(pls)
    specific_int_energy = pls/((L-1)*rls)
    try:
        return (L-1)*rho[0]*specific_int_energy
    except:
        return (L-1)*rho*specific_int_energy # covers case where rho is entered as a single variable.   

def pressure_vs_radius_zaman(kilopascals = False):

    # Using Plank system units (hcut = c = 1)
    # Ref.: http://newfi.narod.ru/Newfi/Constants.htm

    hc = 197.327                # Conversion factor in MeV fm (hut * c)
    G = hc * 6.67259e-45        # Gravitational constant
    Ms = 1.1157467e60
    rho_s = 1665.3              # Central density (density at r = 0)
    M0 = (4*3.14159265*(G**3)*rho_s)**(-0.5)
    R0 = G*M0
    mn = 938.926                # Mass of neutron in MeV c^-2


    # Defining the functions

    # Function for determining initial value of n(r=0)
    N = 4000
    r = np.linspace(0,15,N)
    h = r[1]-r[0]

    m = np.zeros(N)
    p = np.zeros(N)
    rh = np.zeros(N)
    ni = initial_n()

    r[0]   = 0
    m[0] = 0
    p[0] = 363.44 * (ni**2.54)/rho_s
    rh[0] = 1

    flag_set = [0,1]
    print("Initial number density, ni = ", ni)
    print("Initial Pressure, P[0] = ", p[0])
    print("Simulation range, R = 0 to ", r[-1]*R0*1e-18, "km") 
    tol = 9e-5

    # Looping over the two models (k = 0: classical, k = 1: relativistic)
    for k in range(0,2):
        flag = flag_set[k]
        for i in range(0,N-1):
            [m[i+1], p[i+1]] = RK4Solver(r[i],m[i],p[i],h,flag)
            rh[i+1] = rho(r[i])
            if p[i+1] < tol:
                rf = r[i]
                mf = m[i]
                break

        print("Running RK4")
        if i == N-2:
            print("Program didn't converge to P = 0, extend the maximum value of r")
        else:
            print("P <", tol, "found after", i, "runs")

        m = m[0:i+2]
        p = p[0:i+2]
        rh = rh[0:i+2]
        r = r[0:i+2]

    radius_TOV = r*R0*1e-18
    pressure_TOV = p*rho_s
    if kilopascals: # return the pressure in kilopascals
        return radius_TOV, pressure_TOV*1.60218e29 # outputs a radius value in km and pressure value in kilopascals
    return radius_TOV, pressure_TOV # outputs a radius value in km and pressure value in MeV/fm



# **___________________________________saved__________________________**

#### **rho_NS**
Takes in pressure (scalar) and returns density (scalar).

####  **dp_dr** 
Takes in a points mass,pressure, and distance from com. returns differential change in pressure with respect to a differential change in radius at that point. 


####  **dm_dr** 
Takes in a points mass,pressure, and distance from com. returns differential change in mass with respect to a differential change in radius at that point. 
  
 Takes the form $\rho*r^2$
 
 - $M_0 =  (4 \pi G^3 *\rho_{central})^{-0.5}$
- $R_0 = G*M_0$

In [ ]:

#         hc = 197.327           # Conversion factor in MeV fm
#         G = hc * 6.67259e-45   # Gravitational constant
#         Ms = 1.1157467e60      # Mass of the sun in kg
#         mn = 938.926           # Mass of neutron in MeV c^-2
#         self.rho_s = 1665.3 #  Central density (density at r = 0)
#         self.M0 = (4*np.pi*(G**3)*self.rho_s)**(-0.5)
#         self.R0 =  G*self.M0

# def rho_NS(p, rho_s=1665.3): 
#     """ density from pressure NS. """
#     n = (p*rho_s/363.44)**(1./2.54)
#     return (236. * n**2.54 + n *mn)/rho_s

# def dp_dr(r,m,p, Classical = False): 
#     """ pressure dependence on radius NS. """
#     if Classical:                              # classical model
#         return -m*rho_NS(p)/(r**2 + 1e-20)
#     else:                                      # relativistic model                          
#         return -(p+rho_NS(p))*(p*r**3 + m)/(r**2 - 2*m*r + 1e-20)

# def dm_dr(r,m,p):
#     """ Mass dependence on radius (dm/dr)."""
#     return rho_NS(p)*r**2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation

def animate_scatters(iteration, data, scatters):
    """
    For details of this function, see: https://pierresegonne.com/3D_scatterplot/
    """
    for i in range(data[0].shape[0]):
        scatters[i]._offsets3d = (data[iteration][i,0:1], data[iteration][i,1:2], data[iteration][i,2:])
    return scatters


def plot_fig(data, filen, save=True):
    """
    Creates the 3D figure and animates it with the input data.
    Args:
        data (list): List of the data positions at each iteration.
        save (bool): Whether to save the recording of the animation. (Default to False).
    """
    # Attaching 3D axis to the figure
    fig = plt.figure()
    ax = p3.Axes3D(fig)

    # Initialize scatters
    scatters = [ ax.scatter(data[0][i,0:1], data[0][i,1:2], data[0][i,2:]) for i in range(data[0].shape[0]) ]

    # Number of iterations
    iterations = len(data)
    
    axis_len = 20
    # Setting the axes properties
    ax.set_xlim3d([-1*axis_len, axis_len])
    ax.set_xlabel('X')

    ax.set_ylim3d([-1*axis_len, axis_len])
    ax.set_ylabel('Y')

    ax.set_zlim3d([-1*axis_len, axis_len])
    ax.set_zlabel('Z')

    ax.set_title('3D Animated Scatter Example')

    # Provide starting angle for the view.
    ax.view_init(25, 10)

    ani = animation.FuncAnimation(fig, animate_scatters, iterations, fargs=(data, scatters),
                                       interval=50, blit=False, repeat=True)
    if save and filen!=None:
        print("Initializing writer.")
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=30, metadata=dict(artist='Me'), bitrate=1800, extra_args=['-vcodec', 'libx264'])
        print("Saving file.")
        ani.save('animations/'+filen+'.mp4', writer=writer)
        print( "Plot saved to ", 'animations/'+filen+'.mp4')
    plt.show()

In [ ]:
def test_plot_kernels(init = True, keys = ["acc"],ylimits=False, hs=[0.01],ks = ["faber","cubic_spline","wendland_4","wendland_2","gaussian"],times = [(0.001,.02)],num_points=200,plot=False): #,(0.01,.2),(0.0001,.002),(.1,2),(1,20)] # (dt, tEnd)
    KERNELS="-".join(np.copy(ks))
    VARS =""
    ylims = {"acc": (0,10), "dAdt": (0,10),"visc": (0,0.3),"pos":(-12,12),"rho":(0,4000),"grav":(0,0.01),"Pressure":(0,0.01),"(P - grav force)":(0,0.03)}
    ## Test initialization!
    kern_plot = {}
    for h in hs:
        print("\n ******* SUMMARY ******** \n")
        for time in times:
            for k in ks:
                filename="test_get_acc_h=" + str(h) + "dt="+ str(time[0])+ "End=" +str(time[1])
                #eos = EOS("polytropic")
                star1 = NS_("mystar", mass = 1, radius = 12, num_points = num_points,plot_initialized_star=False,kernel_type=k,prt=True)
                if init:
                    pass
                else:
                    star1 = NS_("mystar", mass = 1, radius = 13, num_points = 400,plot_initialized_star=False,kernel_type=kt)
                    star2 = NS_("mysecondstar", mass = 1, radius = 13, num_points = 400,plot_initialized_star=False,kernel_type=kt)
                    star2.com = [8,0,0]
                    star1.com = [-8,0,0]
                    combined_star = star1.combine(star2)
                    print(combined_star.num_points)
                    tEnd = 800
                    print(kt)
                    combined_star.kernel_type = kt
                    com_list = get_com_lists(R = 10)
                    combined_star.plot_init=True
                    combined_star.update_star_new(tag=kt,plotRealTime=True,dt = 0.5,COM_POSITIONS=com_list,params=params)
                kern_plot[k]= star1.test_acc_data
                if plot:
                    plot_single_timestep(star1.pos,title=k+" "+str(h))
        d = star1.test_acc_data
        for key in d.keys():
            if key in keys:
                data = []
                for k in ks:
                    data.append(kern_plot[k][key])   # append data for each kernel
                data = np.asarray(data)
                if ylimits:
                    plt.ylim(ylims[key])
                if key not in ['grav']:
                    csfont = {'fontname':'Times New Roman'}
                    hfont = {'fontname':'Helvetica'}
                    titles= {"acc":"Acceleration","rho":"Density","P":"Pressure","pos":"Position"}
                    k_titles= {"gaussian":"Gaussian","wendland_4":"Wendland 4","wendland_2":"Wendland 2","cubic_spline":"Cubic Spline","faber":"Faber"}
                    print("\nplotting ", titles[key])
                    for i in range(len(data)):
                        plt.plot(data[i])
                        TITLE=titles[key]#
                    plt.title(TITLE,loc='right',**csfont)
                    plt.legend([k_titles[i] for i in ks])
                    plt.ylabel(titles[key])
                    plt.xlabel("Timestep (Code Time)")
                    plt.savefig("plots/"+TITLE+", h="+str(h)+"-"+KERNELS+".jpeg", dpi=150)
                    plt.show()
                else:
                    pass
                if key in []:#["pos"]:
                    print("making movie")
                    plot(data, "pos_NS_anim_"+k)
                
#                     for i in range(len(data)):
#                         plt.plot(data[i])
#                     plt.title(key+",h="+str(h)+", tFin="+str(time[1]),loc='right')
#                     plt.legend(ks)
#                     plt.show()
                print("\n")
    return star1.test_acc_data

In [232]:
def mesh_pts(N,R):
    """ generates N particles inside a sphere mapped onto a uniform grid """
    val = int(np.cbrt(N)) # take cube root of N to get grid side lengths 
    mesh = np.indices((val,val,val)).T.reshape(-1,3) # make a grid with its corner at the origin.
    mesh = mesh/val*2*R  # divide by side length and multiply by 2*R to scale grid to size of star
    mesh = mesh-R # center grid around origin
    r = (mesh[:,0])**2 + (mesh[:,1])**2 + (mesh[:,2])**2
    indices_ = np.where(r>R**2)
    out= np.delete(mesh, list(indices_), axis=0)
    return out